python:3.9.13

tensorflow:2.18.0

----

每局只取1个样本：慢  

缓慢降温增加探索性：效果不好。奖励只在最后一步才出现，探索性破坏奖励信号？  

更小卷积核：测试效果很差  

q-learning：测试效果很差  

sarsa：测试效果很差  

补充人工信息：效果提升

----

目前aug(base4基础上，输入数据增加了人工信息) 和 base4(补上了base2遗漏的batch normalization 和 dropout) 最好, aug 略优

出牌信息里去掉带牌？

----
trainer输出的loss意义有限。如果发牌高度偏向于地主或农民，那么预测就更准，loss就会更小。一轮只有8局，发牌的影响很大。

In [85]:
import numpy as np
import keras as k
import rules
import arena as arn
import bot_bigger
import bot_aug
ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [86]:
def ft(N, bot1_f, bot2_f):
    s = np.zeros((2, N), int)
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot1_f(), bot2_f(), bot2_f()])
        arena.wholegame()
        s[0, i] = arena.winner == 0
        
        arena2 = ARENA(verbos=0, cards=cards.copy())
        arena2.registerbot([bot2_f(), bot1_f(), bot1_f()])
        arena2.wholegame()
        s[1, i] = arena2.winner == 0
        
    # for i in range(N):
    #     arena = ARENA(verbos=0)
    #     arena.registerbot([BOT(model_0), BOT(model), BOT(model)])
    #     # arena.registerbot([BOT(model_0), bot_rd, bot_rd])
    #     arena.wholegame()
    #     s2 += (arena.winner == 0)
    # return s1
    return s

In [87]:
# base2 188150 << 263300 < 308500 ~> 386150 >> 388500 
# aug 284250 ~ 291000 ~ 310850 ~ 323450 < 348450 < 393200 ~ 412500 ~ 422100 ~ 481600 ~ 386850 ~ 471400 ~ 482750 > 452700 
# model = k.models.load_model("model_base2/m{}.keras".format(188150))

model1 = k.models.load_model("model_bigger/m{}.keras".format(82800))
def bot1_f(verbos=0):
    return bot_bigger.BOT(model1, verbos=verbos)

model2 = k.models.load_model("e:/ddz_2025_2_model/model_aug/m{}.keras".format(82800))
def bot2_f(verbos=0):
    return bot_aug.BOT(model2, verbos=verbos)
    # model = k.models.load_model("e:/ddz_2025_2_model/model_base2/m{}.keras".format(386150))
    # return bot_base2.BOT(model, verbos=verbos)

def bot_rd_f():
    return bot_rd

In [88]:
s = ft(100, bot1_f, bot2_f)

In [89]:
print(s.sum(axis=1))
nwins = s.reshape((2, 10, -1)).sum(axis=2)
print((nwins[0] > nwins[1]).sum(), (nwins[0] < nwins[1]).sum(), (nwins[0] == nwins[1]).sum())

[50 48]
5 4 1


In [52]:
np.array([1,2,3,4,5,6]).reshape((2,3)).sum(axis=1)

array([ 6, 15])

In [19]:
arena = arn.ARENA(1, True)
arena.registerbot([bot1_f(1), bot1_f(1), bot1_f(1)])
arena.wholegame()

0.6227842 6 7 8 9 10
0.6200363 3
0.61390114 4
0.610525 9
0.5893781 6 7 8 9 10 J Q K A
 0 :  地主 : 6 7 8 9 10 | 3 4 9 10 10 J Q Q K A 2 2 2 2 小王
0.4004764 
 1 : 农民1 :  | 3 3 4 4 5 5 5 7 8 8 J J J K A A 大王
0.448112 
 2 : 农民2 :  | 3 4 5 6 6 6 7 7 8 9 9 10 Q Q K K A
0.6809657 4
0.6620408 3
0.64314955 10
0.6235338 9 10 J Q K A
0.6060161 Q
 3 :  地主 : 4 | 3 9 10 10 J Q Q K A 2 2 2 2 小王
0.39165717 7
0.33828545 K
0.33767402 5
0.3261258 
0.2636543 A
 4 : 农民1 : 7 | 3 3 4 4 5 5 5 8 8 J J J K A A 大王
0.31259227 9
0.2918104 A
0.28357446 
0.273646 10
0.26813626 8
 5 : 农民2 : 9 | 3 4 5 6 6 6 7 7 8 9 10 Q Q K K A
0.81324065 10
0.80936885 Q
0.6569573 小王
0.51737994 A
0.5076674 2
 6 :  地主 : 10 | 3 9 10 J Q Q K A 2 2 2 2 小王
0.27520263 K
0.16474307 
0.15612411 大王
0.14668232 A
0.12315029 J
 7 : 农民1 : K | 3 3 4 4 5 5 5 8 8 J J J A A 大王
0.20606649 A
0.17843032 
 8 : 农民2 : A | 3 4 5 6 6 6 7 7 8 9 10 Q Q K K
0.84002775 小王
0.6718099 2
0.6519922 
0.25344872 2 2 2 2
 9 :  地主 : 小王 | 3 9 10 J Q Q K A 2 2 2 2
0.14608854 

In [8]:
def bot_mc_f(basebot, verbos=2, RECORD=True, MAXWIDTH=3, NSIMS_FACTOR=50, NSIMS_MAX=1000):
    return mc.BOT(basebot=basebot, verbos=verbos, RECORD=RECORD, MAXWIDTH=MAXWIDTH, NSIMS_FACTOR=NSIMS_FACTOR, NSIMS_MAX=NSIMS_MAX)

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

In [40]:
model1 = k.models.load_model("model_base2/m{}.keras".format(391350))
model2 = k.models.load_model("model_base2/m{}.keras".format(391300))

In [23]:
w1 = model1.weights[1].numpy().reshape(-1)
w2 = model2.weights[1].numpy().reshape(-1)

In [24]:
w1[0:10], w2[0:10]

(array([-0.00342188, -0.00154246, -0.0020585 , -0.00466309, -0.00450408,
         0.06086061,  0.00401902, -0.00338081,  0.00091003,  0.01286665],
       dtype=float32),
 array([-0.0034434 , -0.00159783, -0.00205496, -0.00465686, -0.00447862,
         0.06076964,  0.0040254 , -0.00332928,  0.00091905,  0.01286706],
       dtype=float32))

In [4]:
model_aug = k.models.load_model("model_aug/m{}.keras".format(0))
bot = bot_aug.BOT(model_aug, verbos=1)

c:\Users\BNCD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 30 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
arena = arn.ARENA(1, True)
arena.registerbot([bot, bot, bot])

In [6]:
arena.wholegame()

0.507595 3 6 7 7 7 8 8 8 9 9 9 A
0.50668055 3 6 7 7 7 8 8 8 9 9 9 K
0.5066396 6 6
0.50634277 6 6 7 7 7 8 8 8 9 9
0.5062026 3 7 7 7 8 8 8 9 9 9 K A
 0 :  地主 : 3 6 7 7 7 8 8 8 9 9 9 A | 4 6 J Q K K A 2
0.48654562 10 10 10 10
0.48518962 
 1 : 农民1 : 10 10 10 10 | 5 5 5 6 8 9 J J Q Q K A 大王
0.4915114 
 2 : 农民2 :  | 3 3 3 4 4 4 5 6 7 J Q K A 2 2 2 小王
0.5039667 
 3 :  地主 :  | 4 6 J Q K K A 2
0.49775153 5 5 5 Q Q
0.4963311 5 5 5 Q
0.4963237 5 5 5 J
0.49448997 5 5 5 9
0.49381137 Q Q
 4 : 农民1 : 5 5 5 Q Q | 6 8 9 J J K A 大王
0.5029861 3 3 2 2 2
0.5013083 4 4 2 2 2
0.49796736 
 5 : 农民2 : 3 3 2 2 2 | 3 4 4 4 5 6 7 J Q K A 小王
0.49913236 
 6 :  地主 :  | 4 6 J Q K K A 2
0.49756664 
 7 : 农民1 :  | 6 8 9 J J K A 大王
0.50383705 4 4 4 J
0.50320256 3
0.5031651 J
0.50295943 4 4 4 6
0.50279725 3 4 4 4
 8 : 农民2 : 4 4 4 J | 3 5 6 7 Q K A 小王
0.49925733 
 9 :  地主 :  | 4 6 J Q K K A 2
0.50537 
10 : 农民1 :  | 6 8 9 J J K A 大王
0.50698406 5
0.5058 K
0.5057834 3
0.5042151 6
0.50402355 小王
11 : 农民2 : 5 | 3 6 7 Q K A 小王
0.50

In [25]:
a1 = arena.records[3]
data = bot.getdata(a1)

In [39]:
model1.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 3, 15, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 3, 1, 512)      │       123,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 3, 1, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_10 (ReLU)                 │ (None, 3, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 3, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 1, 1, 512)      │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 1, 1, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,116,933 (19.52 MB)

 Trainable params: 1,703,937 (6.50 MB)

 Non-trainable params: 5,120 (20.00 KB)

 Optimizer params: 3,407,876 (13.00 MB)